In [1]:
from __future__ import print_function
import copy
from threading import Condition, Thread
import numpy as np
import numpy.random as npr
import torch as th
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.modules.loss import CrossEntropyLoss, MSELoss
import torch.nn.functional as F
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader, TensorDataset
import my

In [2]:
import matplotlib.pylab as pl
%matplotlib inline

In [3]:
N_TRAIN, N_TEST = 0, 0
train_data, train_labels, test_data, test_labels = my.unbalanced_cifar10(N_TRAIN, N_TEST, p=[])

train_data_np, train_labels_np, test_data_np, test_labels_np = \
    train_data, train_labels, test_data, test_labels
    
train_data = th.from_numpy(train_data).float()
train_labels = th.from_numpy(train_labels).long()
test_data = th.from_numpy(test_data).float()
test_labels = th.from_numpy(test_labels).long()

cuda = True
if cuda:
    th.cuda.set_device(2)

BATCH_SIZE = 64
train_loader = DataLoader(TensorDataset(train_data, train_labels), BATCH_SIZE, shuffle=True)
test_loader = DataLoader(TensorDataset(test_data, test_labels), BATCH_SIZE)

N_FEATURES = train_data.size()[1]
N_CLASSES = int(train_labels.max() - train_labels.min() + 1)

In [4]:
class CNN(nn.Module):
    def __init__(self, n_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 2, 1)
        self.conv2 = nn.Conv2d(16, 8, 3, 2, 1)
        self.linear = nn.Linear(8, n_classes)
    
    def forward(self, x):
        if x.dim() != 4:
            x = x.view(-1, 3, 32, 32)
        x = F.tanh(self.conv1(x))
        x = F.tanh(self.conv2(x))
        x = F.avg_pool2d(x, 8)
        x = self.linear(x.view(-1, 8))
        return x

In [ ]:
# c = nn.Linear(N_FEATURES, N_CLASSES)
# c = my.MLP((N_FEATURES,) + (64,) * 3 + (N_CLASSES,), F.relu)
# c = CNN(N_CLASSES)
# if cuda:
#     c.cuda()
# optim = Adam(c.parameters(), lr=0.001)
# EPOCHS = 10
# for i in range(EPOCHS):
#     for j, (X, y) in enumerate(train_loader):
#         if cuda:
#             X, y = X.cuda(), y.cuda()
#         X, y = Variable(X), Variable(y)
#         loss = CrossEntropyLoss()(c(X), y)
#         optim.zero_grad()
#         loss.backward()
#         optim.step()
#     accuracy = my.global_stats(c, test_loader, my.accuracy)
#     print('[epoch %d]cross-entropy loss: %f, accuracy: %f' % ((i + 1), float(loss), float(accuracy)))

In [ ]:
# nd_stats = [my.accuracy] + [my.nd_curry(stat, N_CLASSES) for stat in (my.nd_precision, my.nd_recall, my.nd_f_beta)]
# accuracy, precision, recall, f1 = my.global_stats(c, test_loader, nd_stats)
# 'accuracy: %f, precision: %f, recall: %f, f1: %f' % tuple(map(float, (accuracy, precision, recall, f1)))

# Algorithm

Let $c$ be a classifier and $D=\{(X_1, y_1),...,(X_N, y_N)\}$ be the set of training data. In order to minimize $L(c, D)$, where $L$ is a non-decomposable loss function, we introduce $L_\theta$, a parameterized approximation of $L(c, D)$, and update $c$ as follows:

1. Compute $\delta = L(c, D)-L(\bar{c},D)$, where $\bar{c}$ is obtained by stochastically perturbing the parameters of $c$

2. Randomly sample $K$ subsets, $D_1, ..., D_K$, of $D$ (these subsets may vary in cardinality)

3. Minimize $(\delta - \frac1K \sum_{i = 1}^K \delta_i)^2$ with respect to $\theta$, where $\delta_i = L_\theta(c, D_i) - L_\theta(\bar{c}, D_i)$

4. Repeat 1, 2, and 3 several times until $L_\theta$ becomes a satisfactory approximation of $L$ near $c$

5. Randomly sample $K'$ subsets, $D_1, ..., D_K'$, of $D$ and let $c \leftarrow c - \alpha \sum_{i = 1}^K \frac{\partial L_\theta}{\partial c} (c, D_i)$, where $\alpha$ is a positive learning rate

In [5]:
L = lambda c, loader: my.global_stats(c, loader, my.nd_curry(my.nd_f_beta, N_CLASSES))

def forward(classifier, pair):
    x, y = pair
    y = my.onehot(y, N_CLASSES)
    y_bar = F.softmax(classifier(x), 1)
    return th.cat((y, y_bar), 1).view(1, -1)

def sample(sample_size, batch_size):
    # TODO use DataLoader?
    samples = [my.sample_subset(train_data_np, train_labels_np, sample_size) for k in range(batch_size)]
    if cuda:
        samples = [(x.cuda(), y.cuda()) for (x, y) in samples]
    return [(Variable(x), Variable(y)) for (x, y) in samples]

state_dict_cpu2gpu = lambda state_dict: {key : value.cuda() for key, value in state_dict.items()}
state_dict_gpu2cpu = lambda state_dict: {key : value.cpu() for key, value in state_dict.items()}

In [8]:
SAMPLE_SIZE = 16

train_loader = DataLoader(TensorDataset(train_data, train_labels), 1024, shuffle=True)
test_loader = DataLoader(TensorDataset(test_data, test_labels), 1024)

th.random.manual_seed(1)
th.cuda.manual_seed_all(1)

# c = nn.Linear(N_FEATURES, N_CLASSES)
# c = my.MLP((N_FEATURES,) + (512,) * 1 + (N_CLASSES,), F.relu)
c = CNN(N_CLASSES)
critic = my.RN(SAMPLE_SIZE, 2 * N_CLASSES, (2048, 1024, 1024) + (1,), F.relu)

if cuda:
    c.cuda()
    critic.cuda()

# c_optim = SGD(c.parameters(), 0.1, momentum=0.5)
# critic_optim = SGD(critic.parameters(), 0.1, momentum=0.5)
c_optim = Adam(c.parameters(), 1e-3)
critic_optim = Adam(critic.parameters(), 1e-3)

nd_stats = [my.accuracy] + [my.nd_curry(stat, N_CLASSES) for stat in (my.nd_precision, my.nd_recall, my.nd_f_beta)]
accuracy, precision, recall, f1 = my.global_stats(c, test_loader, nd_stats)
'accuracy: %f, precision: %f, recall: %f, f1: %f' % tuple(map(float, (accuracy, precision, recall, f1)))
# TODO unusual precision

'accuracy: 0.100000, precision: 0.010000, recall: 0.100000, f1: 0.018180'

In [ ]:
std = 1e-1
tau = 1e-2

N_ITERATIONS = 100
N_PERTURBATIONS = 50
CRITIC_ITERATIONS = 10
ACTOR_ITERATIONS = 5
BATCH_SIZE = 1

hist = []
for i in range(N_ITERATIONS):
    hist.append({})
    hist[-1]['c_state_dict'] = copy.deepcopy(state_dict_gpu2cpu(c.state_dict()))
    
    c.eval()
    critic.train()
    L_c = L(c, test_loader)
#     L_c = L(c, train_loader)
    c_bar_list, t_list = [], []
    f1_list = []
    for j in range(N_PERTURBATIONS):
        c_bar_list.append(my.perturb(c, std))
        L_bar = L(c_bar_list[-1], test_loader)
#         L_bar = L(c_bar_list[-1], train_loader)
        f1_list.append(float(L_bar))
        t = L_c - L_bar
        t_list.append(t[0])
    hist[-1]['f1_list'] = f1_list
    w_list = [th.exp(-t ** 2 / tau) for t in t_list] # TODO proposal distribution
    z = sum(w_list)
    w_list = [(w / z).detach() for w in w_list]
    hist[-1]['w_list'] = w_list
    
    s = sample(SAMPLE_SIZE, BATCH_SIZE)
    hist[-1]['s'] = s
    y = th.cat([forward(c, x) for x in s], 0).detach()
    for j in range(CRITIC_ITERATIONS):
        for c_bar, t, w in zip(c_bar_list, t_list, w_list):
            y_bar = th.cat([forward(c_bar, x) for x in s], 0).detach()
            delta = th.mean(critic(y) - critic(y_bar), 0)
            mse = w * MSELoss()(delta, t)
            critic_optim.zero_grad()
            mse.backward()
            critic_optim.step()
    hist[-1]['critic_state_dict'] = copy.deepcopy(state_dict_gpu2cpu(critic.state_dict()))

    c.train()
    critic.eval()
    c_parameters = copy.deepcopy(tuple(c.parameters()))
    for j in range(ACTOR_ITERATIONS):
        y = th.cat([forward(c, x) for x in s], 0)
        objective = -th.mean(critic(y)) # TODO critic-awareness
        c_optim.zero_grad()
        objective.backward()
        c_optim.step()
        if any(float(th.max(th.abs(p - q))) > std for p, q in zip(c_parameters, c.parameters())):
            break

    if (i + 1) % 1 == 0:
        f1 = my.global_stats(c, test_loader, my.nd_curry(my.nd_f_beta, N_CLASSES))
        hist[-1]['f1'] = float(f1)
        print('[iteration %d]f1: %f' % (i + 1, f1))

[iteration 1]f1: 0.042485
[iteration 2]f1: 0.041974
[iteration 3]f1: 0.059990
[iteration 4]f1: 0.061109
[iteration 5]f1: 0.077854
[iteration 6]f1: 0.084579
[iteration 7]f1: 0.104773


In [ ]:
# h = hist[-1]
# c, c_bar = CNN(N_CLASSES), CNN(N_CLASSES)
# c.load_state_dict(h['c_state_dict'])
# c_bar.load_state_dict(h['c_state_dict'])
# critic = my.RN(SAMPLE_SIZE, 2 * N_CLASSES, (1024,) * 3 + (1,), F.relu)
# critic.load_state_dict(h['critic_state_dict'])
# if cuda:
#     c.cuda()
#     c_bar.cuda()
#     critic.cuda()

# # TODO gradient in parameter space/simplex
# y = th.cat([forward(c, x) for x in h['s']], 0)
# objective = -th.mean(critic(y))
# objective.backward()

# alpha = np.linspace(0, 1e1)
# f1_list = []
# for a in alpha:
#     for p, p_bar in zip(c.parameters(), c_bar.parameters()):
#         p_bar.data = p.data - float(a) * p.grad.data
#     f1_list.append(float(L(c_bar, test_loader)))

# pl.plot(alpha, f1_list)

In [ ]:
f1_list = [np.array(list(map(float, h['f1_list']))) for h in hist]
min_list = tuple(map(np.min, f1_list))
max_list = tuple(map(np.max, f1_list))
std_list = tuple(map(np.std, f1_list))
pl.plot(range(len(f1_list)), min_list, label='min sample')
pl.plot(range(len(f1_list)), max_list, label='max sample')
pl.plot(range(len(hist)), [h['f1'] for h in hist], label='classifier')
pl.title('f1')
pl.legend(framealpha=0)
pl.figure()
pl.title('std')
pl.plot(range(len(f1_list)), std_list)

In [ ]:
entropy_list = []
for h in hist:
    w_array = np.array(list(map(float, h['w_list'])))
    entropy_list.append(-np.sum(w_array * np.log(w_array)))
n = len(hist)
pl.plot(range(n), entropy_list, label='importance distribution')
pl.plot(range(n), -np.ones(n) * np.log(1 / N_PERTURBATIONS), label='uniform distribution')
pl.title('entropy')
pl.legend(framealpha=0)

In [ ]:
w_list = [np.array(list(map(float, h['w_list']))) for h in hist]
min_list = tuple(map(np.min, w_list))
max_list = tuple(map(np.max, w_list))
std_list = tuple(map(np.std, w_list))
pl.plot(range(len(w_list)), min_list, label='min')
pl.plot(range(len(w_list)), max_list, label='max')
pl.title('importance weight')
pl.legend(framealpha=0)
pl.figure()
pl.title('importance weight')
pl.plot(range(len(w_list)), std_list, label='std')
pl.legend(framealpha=0)

In [ ]:
# critic for mnist

# N_ITERATIONS = 25
# BATCH_SIZE = 8

# critic.load_state_dict(th.load('mnist_critic'))
# for i in range(N_ITERATIONS):
#     s = sample(SAMPLE_SIZE, BATCH_SIZE)
#     y = th.cat([forward(c, x) for x in s], 0)
#     objective = -th.mean(critic(y))
#     c_optim.zero_grad()
#     objective.backward()
#     c_optim.step()
        
#     if (i + 1) % 1 == 0:
#         f1 = my.global_stats(c, test_loader, my.nd_curry(my.nd_f_beta, N_CLASSES))
#         print('[iteration %d]f1: %f' % (i + 1, f1))

In [ ]:
# TODO parallel sampling

# gpus = [0, 1, 2, 3]
# train_data_dict = {gpu: train_data.cuda(gpu) for gpu in gpus}
# train_labels_dict = {gpu: train_labels.cuda(gpu) for gpu in gpus}

# def L_batch(c, std, n):
#     c_device = next(c.parameters()).get_device()
#     c = deepcopy(c).cpu()
#     c.train(False)
#     c_dict = {gpu: deepcopy(c).cuda(gpu) for gpu in gpus}
#     results = []
#     available_gpus = [gpu for gpu in gpus]
#     condition = Condition()
    
#     def target(gpu):
#         c, train_data, train_labels = c_dict[gpu], train_data_dict[gpu], train_labels_dict[gpu]
#         c_bar = my.perturb(c, std)
        
#         delta = L(c, train_data, train_labels) - L(c_bar, train_data, train_labels) # TODO
#         results.append((c_bar.cuda(c_device), delta.cuda(c_device).detach()))
#         with condition:
#             available_gpus.append(gpu)
#             condition.notify_all()
    
#     threads = []
#     with condition:
#         for i in range(n):
#             if not available_gpus:
#                 condition.wait()
#             gpu = available_gpus.pop()
#             threads.append(Thread(target=target, args=(gpu,)))
#             threads[-1].start()
#     for t in threads:
#         t.join()
#     return results

In [ ]:
# TODO parallel sampling

# STD = 0.05
# OUTER = 5000
# INNER_ACTOR = 5
# INNER_CRITIC = 20

# mse_list = []
# for i in range(OUTER):
#     targets = L_batch(c, STD, INNER_CRITIC)
#     for j in range(INNER_CRITIC):
#         c_bar, delta = targets[j]
#         samples = sample()
#         y = th.cat(tuple(map(lambda x: forward(c, x), samples)), 0).detach()
#         y_bar = th.cat(tuple(map(lambda x: forward(c_bar, x), samples)), 0).detach()
        
#         mse = 1
#         while float(mse) > 1e-3:
#             delta_ = th.mean(critic(y) - critic(y_bar), 0)
#             mse = MSELoss()(delta_, delta)
#             mse_list.append(mse)
#             critic_optim.zero_grad()
#             mse.backward()
#             critic_optim.step()

#     c_parameters = deepcopy(tuple(c.parameters()))
#     for j in range(INNER_ACTOR):
#         samples = sample()
#         y = th.cat(tuple(map(lambda x: forward(c, x), samples)), 0)
#         objective = -th.mean(critic(y))
#         c_optim.zero_grad()
#         objective.backward()
#         c_optim.step()
#         if any(float(th.max(th.abs(p - q))) > STD for p, q in zip(c_parameters, c.parameters())):
#             break

#     if (i + 1) % 1 == 0:
#         y_bar = my.predict(c, test_data)
#         f1 = my.nd_f_beta(y_bar, test_labels, N_CLASSES)
#         print('[iteration %d]mse: %f, objective: %f, f1: %f' % ((i + 1), float(mse), float(objective), float(f1)))

In [ ]:
# TODO F_beta implementation

# from sklearn.metrics import precision_score, recall_score, f1_score
# y_bar = my.predict(c, test_data).data.cpu().numpy()
# precision_score(test_labels_np, y_bar, average='macro'), \
# recall_score(test_labels_np, y_bar, average='macro'), \
# f1_score(test_labels_np, y_bar, average='macro')